В данном ноутбуке описан алгоритм предсказания на основе ввода пользователя

In [178]:
year = int(input('Год до которого предсказывать:'))
month = int(input('Месяц выбранного года до которого предсказывать:'))
print(f'{month}.{year}')

1.2025


In [225]:
import pandas as pd
import numpy as np
import pickle

In [226]:
from sklearn.preprocessing import LabelEncoder

iso_encoder = LabelEncoder()
iso_encoder.classes_ = np.load('..\models\iso_encoder_classes.npy')

In [227]:
statistics_models = [
    {
        'name': 'consumer_price',
        "model": pickle.load(open('..\models\countries_consumer_price.sav', 'rb')),
    },
    {
        'name': 'populations',
        "model": pickle.load(open('..\models\countries_populations.sav', 'rb')),
    },
    {
        'name': 'gdp',
        "model": pickle.load(open('..\models\countries_gdp.sav', 'rb')),
    }
]

In [228]:
cars_sales_training =  pd.read_csv('..\data\cars_sales_training.csv', index_col=0)

In [229]:
max_year =cars_sales_training.sort_values(by=['year', 'month'], ascending=False).iloc[0].year
max_month =cars_sales_training.sort_values(by=['year', 'month'], ascending=False).iloc[0].month
max_year, max_month

(2023, 9)

In [230]:
prediction_dates = pd.date_range(start=f'{max_month+1}/1/{max_year}', end=f'{month}/1/{year}', freq='MS').to_list()

In [231]:
countries = cars_sales_training['iso_alpha'].unique()

In [249]:
prediction_df = pd.DataFrame().reindex_like(cars_sales_training).dropna()
prediction_df

,year,month,sales,iso_alpha,consumer_price,populations,gdp


In [250]:
for country_code in countries:
    for date in prediction_dates:
        data = {
            'year' : date.year,
            'month' : date.month,
            'iso_alpha': country_code,
        }
        if date.month == 1:
            for datamodel in statistics_models:
                pred_data =  pd.DataFrame([{'iso_alpha': country_code, 'year': date.year }])
                pred_data['iso_alpha'] = iso_encoder.transform(pred_data['iso_alpha'])
                data[datamodel['name']] = datamodel['model'].predict(pred_data)[0]
        new_row = pd.DataFrame([data])
        prediction_df = pd.concat([prediction_df, new_row], ignore_index = True) 
            


                    
    

In [251]:
statistics_models[0]['model'].predict(pd.DataFrame([{'iso_alpha': iso_encoder.transform(['JPN'])[0], 'year': 2022 }]))

array([172.74233478])

In [252]:
prediction_df = pd.concat([cars_sales_training, prediction_df], ignore_index = True,) 
prediction_df[prediction_df['iso_alpha']== 'JPN'].tail(25)

,year,month,sales,iso_alpha,consumer_price,populations,gdp
3417,2023.0,1.0,379637.0,JPN,176.415799,3.790251e+07,4.041552e+11
3418,2023.0,2.0,422534.0,JPN,176.415799,3.790251e+07,4.041552e+11
3419,2023.0,3.0,566470.0,JPN,176.415799,3.790251e+07,4.041552e+11
3420,2023.0,4.0,345784.0,JPN,176.415799,3.790251e+07,4.041552e+11
3421,2023.0,5.0,322886.0,JPN,176.415799,3.790251e+07,4.041552e+11
3422,2023.0,6.0,387445.0,JPN,176.415799,3.790251e+07,4.041552e+11
3423,2023.0,7.0,374907.0,JPN,176.415799,3.790251e+07,4.041552e+11
3424,2023.0,8.0,335873.0,JPN,176.415799,3.790251e+07,4.041552e+11
3425,2023.0,9.0,432237.0,JPN,176.415799,3.790251e+07,4.041552e+11
7269,2023.0,10.0,NaN,JPN,NaN,NaN,NaN


In [253]:
prediction_df.sort_values(by=['iso_alpha', 'year', 'month'], inplace=True)

In [254]:
for country_code in countries:
    for datamodel in statistics_models:
        prediction_df.loc[prediction_df[prediction_df['iso_alpha'] == country_code].index, datamodel['name']] = prediction_df[prediction_df['iso_alpha'] == country_code][datamodel['name']].interpolate(method='linear',limit_direction='both', limit=50)

In [255]:
prediction_df[prediction_df['iso_alpha']== 'JPN'].tail(25)

,year,month,sales,iso_alpha,consumer_price,populations,gdp
3417,2023.0,1.0,379637.0,JPN,176.415799,3.790251e+07,4.041552e+11
3418,2023.0,2.0,422534.0,JPN,176.415799,3.790251e+07,4.041552e+11
3419,2023.0,3.0,566470.0,JPN,176.415799,3.790251e+07,4.041552e+11
3420,2023.0,4.0,345784.0,JPN,176.415799,3.790251e+07,4.041552e+11
3421,2023.0,5.0,322886.0,JPN,176.415799,3.790251e+07,4.041552e+11
3422,2023.0,6.0,387445.0,JPN,176.415799,3.790251e+07,4.041552e+11
3423,2023.0,7.0,374907.0,JPN,176.415799,3.790251e+07,4.041552e+11
3424,2023.0,8.0,335873.0,JPN,176.415799,3.790251e+07,4.041552e+11
3425,2023.0,9.0,432237.0,JPN,176.415799,3.790251e+07,4.041552e+11
7269,2023.0,10.0,NaN,JPN,177.334165,3.799691e+07,4.060805e+11


In [256]:
sales_prediction_model = pickle.load(open('..\models\car_sales.sav', 'rb'))

In [257]:
pred_x = prediction_df[prediction_df.isna().any(axis=1)].drop(columns='sales')
pred_x['iso_alpha'] = iso_encoder.transform(pred_x['iso_alpha'])
pred_x

,year,month,iso_alpha,consumer_price,populations,gdp
6789,2023.0,10.0,7,152.395080,1.869363e+07,4.747108e+11
6790,2023.0,11.0,7,154.231812,2.794614e+07,4.423576e+11
6791,2023.0,12.0,7,156.068544,3.719864e+07,4.100044e+11
6792,2024.0,1.0,7,157.905276,4.645114e+07,3.776513e+11
6793,2024.0,2.0,7,158.211398,4.648261e+07,3.782930e+11
...,...,...,...,...,...,...
7792,2024.0,9.0,246,209.446593,2.862070e+07,4.584796e+11
7793,2024.0,10.0,246,209.752715,2.865217e+07,4.591214e+11
7794,2024.0,11.0,246,210.058837,2.868363e+07,4.597631e+11
7795,2024.0,12.0,246,210.364959,2.871510e+07,4.604049e+11


In [258]:
prediction_df.loc[prediction_df[prediction_df.isna().any(axis=1)].index, 'sales'] = sales_prediction_model.predict(pred_x)

In [259]:
prediction_df[prediction_df['iso_alpha'] == 'ZAF' ].tail(25)

,year,month,sales,iso_alpha,consumer_price,populations,gdp
6780,2023.0,1.0,42262.000000,ZAF,203.324153,2.799135e+07,4.456446e+11
6781,2023.0,2.0,43130.000000,ZAF,203.324153,2.799135e+07,4.456446e+11
6782,2023.0,3.0,47542.000000,ZAF,203.324153,2.799135e+07,4.456446e+11
6783,2023.0,4.0,35515.000000,ZAF,203.324153,2.799135e+07,4.456446e+11
6784,2023.0,5.0,41102.000000,ZAF,203.324153,2.799135e+07,4.456446e+11
6785,2023.0,6.0,44538.000000,ZAF,203.324153,2.799135e+07,4.456446e+11
6786,2023.0,7.0,41558.000000,ZAF,203.324153,2.799135e+07,4.456446e+11
6787,2023.0,8.0,43502.000000,ZAF,203.324153,2.799135e+07,4.456446e+11
6788,2023.0,9.0,43880.000000,ZAF,203.324153,2.799135e+07,4.456446e+11
7781,2023.0,10.0,25571.141910,ZAF,204.242519,2.808576e+07,4.475699e+11
